References  
1. <https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/>  
1. <http://www.engear.net/wp/topic-modeling-gensimpython/>
1. <https://www.lucypark.kr/courses/2015-ba/text-mining.html>  
1. <https://github.com/smilemango/lda_study>  

In [2]:
!pip install gensim
!pip install pyLDAvis
!pip install spacy

In [3]:
!pip install nltk
!pip install stopwords

    100% |████████████████████████████████| 51kB 4.5MB/s 
  Running setup.py bdist_wheel for stopwords ... - \ done
  Stored in directory: /root/.cache/pip/wheels/39/fa/c7/c4c5111e658f5c58465d948165dc3395a3c10ff57f4cd20356
Successfully built stopwords


In [4]:
!git clone https://github.com/yunho0130/lda_study

Cloning into 'lda_study'...
remote: Enumerating objects: 6, done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 6
Unpacking objects: 100% (6/6), done.


In [0]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [7]:
# Import Dataset
df = pd.read_csv("hmd_vr_reviews.csv")
# comments
print(df.comments.unique())
df.head()

['No me funcion??' 'Did not work!!!' 'kids enjoy them' ...
 'I bought this as an impulse after a friend brought his over and we played around on it for a couple hours. Here is my in breakdown if you don\'t want to read the entire review.Pro\'s:1. Fun for getting a taste of where the VR technology is going and what will be available in a few years.2. Compared to other options for price and quality available (Cardboard, Vive, VR One, Oculus) it is affordable and *easy to use.3. Decent look overall and comfortable to wear.* Device hardware is simple enough but the software could use some work. For instance you would not be able to hand this over to someone and them intuitively figure out how to navigate the system easily which means you will basically have to have the person take off the headset each time to load up the next "demo" you want to show them.Con\'s:1. Uses phone as processing source which means you can\'t use phone while in operation.2. Tends to overheat after 45mins-1hour+ of

,senti_score,deviceName,deviceCategory,reviewNo,reviewDivision,reviewUrl,helpResponceCnt,reviewWriter,date,purchase,...,PRQ02,PRQ03,TCT01,TCT02,TCT03,PCC01,PCC02,CYS01,CYS02,CYS03
0,-0.876485,Samsung-Gear-VR,Gear VR,1078,critical,http://www.amazon.com/gp/customer-reviews/R3MD...,0,Amazon Customer,12-Feb-16,Verified Purchase,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.663209,View-Master,AdvancedCardboard,571,critical,http://www.amazon.com/gp/customer-reviews/R2WS...,1,M S,05-Jan-16,Verified Purchase,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.000000,DESTEK-Distance-Adjustable,AdvancedCardboard,249,critical,http://www.amazon.com/gp/customer-reviews/R3UF...,0,Mark D Jessen,09-Nov-15,Verified Purchase,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.000000,Google-Cardboard,Cardboard,1156,critical,http://www.amazon.com/gp/customer-reviews/R1SD...,0,Michael Herman,01-Feb-16,Verified Purchase,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.000000,IncrediSonic M700,AdvancedCardboard,315,positive,http://www.amazon.com/gp/customer-reviews/R15O...,1,Amazon Customer,09-Nov-15,Verified Purchase,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
import os
# Please upload csv file after setting chdir
os.chdir('lda_study')

In [9]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [12]:
# Convert to list
data = df.comments.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['No me funcion??']


In [13]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['no', 'me', 'funcion']]


In [14]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['no', 'me', 'funcion']


In [0]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [16]:
!python3 -m spacy download en

    100% |████████████████████████████████| 37.4MB 81.3MB/s 
  Running setup.py install for en-core-web-sm ... - \ | / done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [17]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['funcion']]


In [18]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1)]]


In [19]:
id2word[0]

'funcion'

In [20]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('funcion', 1)]]

In [0]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [22]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.071*"first" + 0.063*"hold" + 0.043*"keep" + 0.036*"back" + '
  '0.035*"however" + 0.031*"different" + 0.028*"technology" + 0.028*"open" + '
  '0.024*"vr" + 0.024*"top"'),
 (1,
  '0.181*"product" + 0.163*"would" + 0.088*"great" + 0.071*"need" + '
  '0.050*"content" + 0.037*"enough" + 0.035*"recommend" + 0.034*"nice" + '
  '0.026*"star" + 0.020*"other"'),
 (2,
  '0.079*"see" + 0.075*"come" + 0.069*"take" + 0.065*"head" + 0.048*"side" + '
  '0.048*"strap" + 0.044*"goggle" + 0.034*"may" + 0.023*"velcro" + '
  '0.022*"world"'),
 (3,
  '0.212*"really" + 0.106*"download" + 0.099*"cool" + 0.079*"something" + '
  '0.049*"opinion" + 0.033*"fold" + 0.021*"year" + 0.020*"walk" + 0.018*"cost" '
  '+ 0.018*"effect"'),
 (4,
  '0.093*"time" + 0.048*"nose" + 0.043*"bit" + 0.038*"amazing" + 0.035*"focus" '
  '+ 0.034*"long" + 0.031*"image" + 0.031*"pad" + 0.028*"magnet" + '
  '0.022*"simply"'),
 (5,
  '0.140*"try" + 0.102*"device" + 0.058*"have" + 0.036*"definitely" + '
  '0.029*"simple" + 0.0

In [23]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.193229615138423

Coherence Score:  0.27652537881797573


In [24]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
15     8.498781        1       1 -0.286009  0.192516
8      8.122121        1       2 -0.149974 -0.364086
6      7.096648        1       3 -0.133770 -0.117643
13     6.881136        1       4 -0.231093  0.173251
2      6.213256        1       5 -0.012689 -0.081254
9      5.810004        1       6 -0.012537 -0.020588
16     5.425231        1       7  0.015402 -0.015765
14     5.218257        1       8  0.020304 -0.001898
1      4.986952        1       9  0.007595 -0.030726
0      4.873808        1      10  0.031630 -0.002317
7      4.787095        1      11  0.034903 -0.035589
4      4.568716        1      12  0.044217  0.015692
11     3.951574        1      13  0.066351  0.030868
10     3.866448        1      14  0.074860  0.028845
5      3.797835        1      15  0.079099  0.036593
17     3.740793        1      16  0.070568  0.030038
19     3.569980        1      17  0.080214  0.041628
18     3.106558        1      18  0.091846  0.041905
3      2.983991        1      19  0.097099  0.040228
12     2.500816        1      20  0.111984  0.038301, topic_info=     Category         Freq         Term        Total  loglift  logprob
term                                                                  
99    Default  7074.000000        phone  7074.000000  30.0000  30.0000
219   Default  6293.000000    cardboard  6293.000000  29.0000  29.0000
83    Default  5387.000000          not  5387.000000  28.0000  28.0000
35    Default  4749.000000          app  4749.000000  27.0000  27.0000
11    Default  3875.000000      product  3875.000000  26.0000  26.0000
189   Default  4517.000000          get  4517.000000  25.0000  25.0000
447   Default  3478.000000        would  3478.000000  24.0000  24.0000
448   Default  3344.000000      headset  3344.000000  23.0000  23.0000
39    Default  2713.000000       really  2713.000000  22.0000  22.0000
15    Default  3074.000000   experience  3074.000000  21.0000  21.0000
280   Default  3123.000000          use  3123.000000  20.0000  20.0000
111   Default  3019.000000       review  3019.000000  19.0000  19.0000
27    Default  2696.000000         make  2696.000000  18.0000  18.0000
581   Default  2348.000000           go  2348.000000  17.0000  17.0000
227   Default  2372.000000         gear  2372.000000  16.0000  16.0000
16    Default  3065.000000         good  3065.000000  15.0000  15.0000
61    Default  2199.000000        glass  2199.000000  14.0000  14.0000
418   Default  2282.000000          try  2282.000000  13.0000  13.0000
82    Default  2993.000000           do  2993.000000  12.0000  12.0000
1     Default  2718.000000         work  2718.000000  11.0000  11.0000
37    Default  2888.000000           be  2888.000000  10.0000  10.0000
65    Default  2359.000000         look  2359.000000   9.0000   9.0000
256   Default  2188.000000        video  2188.000000   8.0000   8.0000
268   Default  2457.000000         game  2457.000000   7.0000   7.0000
330   Default  2311.000000         also  2311.000000   6.0000   6.0000
226   Default  2108.000000          see  2108.000000   5.0000   5.0000
64    Default  1788.000000        thing  1788.000000   4.0000   4.0000
632   Default  2274.000000      virtual  2274.000000   3.0000   3.0000
203   Default  1831.000000         time  1831.000000   2.0000   2.0000
13    Default  1899.000000        great  1899.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
340   Topic20   781.192139            s   781.423401   3.6883  -2.6199
14    Topic20   526.278198     purchase   526.509460   3.6881  -3.0149
490   Topic20   443.265228  application   443.496490   3.6880  -3.1865
1189  Topic20   312.011658       degree   312.242920   3.6878  -3.5377
341   Topic20   288.292938         that   288.524200   3.6878  -3.6167
98    Topic20   287.893097       future   288.124359   3.6878  -3.6181
193   Topic20   263.45